In [ ]:
import gc
import momepy
import osmnx as ox
import networkx as nx
from datetime import datetime
import geopandas as gpd
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# xmin, ymin, xmax, ymax = 115.898895, 39.620499, 116.845093, 40.194610
xmin, ymin, xmax, ymax = 116.333714, 39.937053, 116.437740, 39.985736

In [ ]:
%%time
# gdf = gpd.read_parquet(r'C:\Projects\data\geolife\geolife_lines.parquet')
print(len(gdf))

In [ ]:
%%time
gdf = gdf.cx[xmin:xmax, ymin:ymax]
print(len(gdf))

In [ ]:
%%time
G = momepy.gdf_to_nx(gdf, approach="primal")

In [ ]:
%%time
nodes, edges = momepy.nx_to_gdf(G)

In [ ]:
%%time
nodes['x'], nodes['y'] = nodes.geometry.x, nodes.geometry.y

In [ ]:
%%time
edges['u'] = edges['node_start']
edges['v'] = edges['node_end']
edges['k'] = 0
edges.set_index(['u','v','k'], inplace=True)

In [ ]:
%%time
oxG = ox.utils_graph.graph_from_gdfs(nodes, edges, graph_attrs=None)

In [ ]:
%%time
oxG_simplified = ox.simplify_graph(oxG)

In [ ]:
%%time
oxG_proj = ox.project_graph(oxG_simplified)

In [12]:
%%time #way to slow for dataset - going to try to use a grid to rasterize the lines then skeletonize and convert to a graph that way
G2 = ox.consolidate_intersections(oxG_proj, rebuild_graph=True, tolerance=15, dead_ends=False)

In [ ]:
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G2, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True)

In [ ]:
gdf_nodes.to_parquet(r'C:\Projects\data\geolife\geolife_graph_nodes.parquet')
gdf_edges.to_parquet(r'C:\Projects\data\geolife\geolife_graph_edges.parquet')

In [ ]:
# fig, ax = ox.plot_graph(G2, node_color="r")